In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 761 ms (started: 2021-08-26 18:36:55 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #7  Aug 26, 2021'

#git push origin master
git push

In [2]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[1] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 5.29 s (started: 2021-08-26 18:37:11 +08:00)


# 介绍
TensorFlow Cloud 是一个库，可让您更轻松地在 Google Cloud 上对 Keras 模型进行训练和超参数调整。

使用 TensorFlow Cloud 的运行 API，您可以将模型代码直接发送到您的 Google Cloud 帐户，并使用 Google Cloud 计算资源而无需登录和与 Cloud UI 交互（一旦您在控制台中设置了您的项目）。

这意味着您可以直接从 Python 笔记本内部使用您的 Google Cloud 计算资源：就像这个笔记本一样！ 您还可以从一个普通的 .py Python 脚本将模型发送到 GCP。

# 简单的例子
这是一个简单的介绍性示例，用于演示如何使用 TensorFlow Cloud 和 Google Cloud 远程训练模型。

您可以通读一遍以了解其工作原理，也可以在 Google Colab 中运行该笔记本。 运行笔记本需要连接到 Google Cloud 帐户并输入您的凭据和项目 ID。 如果您还没有帐户或不确定如何在控制台中设置项目，请参阅设置并连接到您的 Google Cloud 帐户。

# 导入所需模块
本指南需要 TensorFlow Cloud，您可以通过以下方式安装：

In [ ]:
pip install tensorflow-cloud

# 项目配置
设置项目参数。 如果您不知道 `GCP_PROJECT_ID` 或 `GCS_BUCKET` 应该是什么，请参阅设置并连接到您的 Google Cloud 帐户。

`JOB_NAME` 是可选的，您可以将其设置为任何字符串。 如果您正在做多个培训实验（例如）作为较大项目的一部分，您可能希望为每个实验都指定一个唯一的 JOB_NAME。

In [ ]:
# Set Google Cloud Specific parameters

# TODO: 请将 GCP_PROJECT_ID 设置为您自己的 Google Cloud 项目 ID。
GCP_PROJECT_ID = "YOUR_PROJECT_ID"  # @param {type:"string"}

# TODO: 将 GCS_BUCKET 设置为您自己的 Google Cloud Storage (GCS) 存储分区。
GCS_BUCKET = "YOUR_GCS_BUCKET_NAME"  # @param {type:"string"}

# 请勿更改：目前仅支持“us-central1”区域。
REGION = "us-central1"

# 可选：您可以将作业名称更改为任何字符串。
JOB_NAME = "mnist"  # @param {type:"string"}

# 设置位置是训练日志和检查点将被存储
GCS_BASE_PATH = f"gs://{GCS_BUCKET}/{JOB_NAME}"
TENSORBOARD_LOGS_DIR = os.path.join(GCS_BASE_PATH, "logs")
MODEL_CHECKPOINT_DIR = os.path.join(GCS_BASE_PATH, "checkpoints")
SAVED_MODEL_DIR = os.path.join(GCS_BASE_PATH, "saved_model")

# 对Notebook进行身份验证以使用您的 Google Cloud 项目
此代码对笔记本进行身份验证，检查您的有效 Google Cloud 凭据和身份。 它位于 `if not tfc.remote()` 块内，以确保它仅在笔记本中运行，并且在将笔记本代码发送到 Google Cloud 时不会运行。

注意：对于 Kaggle Notebooks，在运行下面的单元之前点击“附加组件”->“Google Cloud SDK”。

In [ ]:
# Using tfc.remote() to ensure this code only runs in notebook
if not tfc.remote():

    # Authentication for Kaggle Notebooks
    if "kaggle_secrets" in sys.modules:
        from kaggle_secrets import UserSecretsClient

        UserSecretsClient().set_gcloud_credentials(project=GCP_PROJECT_ID)

    # Authentication for Colab Notebooks
    if "google.colab" in sys.modules:
        from google.colab import auth

        auth.authenticate_user()
        os.environ["GOOGLE_CLOUD_PROJECT"] = GCP_PROJECT_ID

# 模型和数据设置
从这里开始，我们将遵循设置简单 Keras 模型以在 MNIST 数据集上运行分类的基本过程。

## 加载和拆分数据
读取原始数据并拆分以训练和测试数据集。

In [4]:
(x_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape((60000, 28 * 28))
x_train = x_train.astype("float32") / 255

time: 28.3 ms (started: 2021-08-26 18:53:45 +08:00)


# 创建模型并准备训练
创建一个简单的模型并为其设置一些回调。

In [ ]:
from tensorflow.keras import layers

model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(512, activation="relu", input_shape=(28 * 28,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

## 快速验证培训
我们将对模型进行一 (1) 次训练，以确保一切设置正确，如果不是 tfc.remote，我们会将训练命令包装在其中，以便它仅在您所在的运行时环境中发生 正在阅读此内容，而不是将其发送到 Google Cloud 时。

# 准备远程培训
下面的代码只会在笔记本代码发送到 Google Cloud 时运行，而不是在您阅读本文的运行时内运行。

首先，我们设置回调，它将：
* 为 TensorBoard 创建日志。
* 创建检查点并将它们保存到上面指定的检查点目录。
* 如果损失没有得到充分改善，请停止模型训练。

然后我们调用 model.fit 和 model.save，它们（当此代码在 Google Cloud 上运行时）实际运行完整的训练（100 个 epoch），然后将训练好的模型保存在上面定义的 GCS Bucket 和目录中。

In [ ]:
if tfc.remote():
    # Configure Tensorboard logs
    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=TENSORBOARD_LOGS_DIR),
        tf.keras.callbacks.ModelCheckpoint(MODEL_CHECKPOINT_DIR, save_best_only=True),
        tf.keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.001, patience=3),
    ]

    model.fit(
        x=x_train, y=y_train, epochs=100, validation_split=0.2, callbacks=callbacks,
    )

    model.save(SAVED_MODEL_DIR)

# 开始远程培训
TensorFlow Cloud 从其本地执行环境（此笔记本）中获取所有代码，将其打包，然后将其发送到 Google Cloud 以执行。 （这就是 if 和 if not tfc.remote 包装器很重要的原因。）

此步骤将从该笔记本准备您的代码以供远程执行，然后在 Google Cloud Platform 上启动远程训练作业以训练模型。

首先，我们将 tensorflow-cloud Python 包添加到 requirements.txt 文件中，该文件将与本笔记本中的代码一起发送。您可以根据需要在此处添加其他软件包。

然后指定一个 GPU 和一个 CPU 图像。您只需要指定其中之一；下面的代码中使用了 GPU。

最后，TensorFlow 云的核心：调用 tfc.run。当这个在这个 notebook 中执行时，这个 notebook 中的所有代码，以及这个目录中的其余文件，都将被打包并发送到谷歌云执行。 run 方法上的参数指定了 GPU CPU 图像的详细信息。您只需要指定其中之一；下面的代码中使用了 GPU。

最后，TensorFlow 云的核心：调用 tfc.run。 当这个在这个 notebook 中执行时，这个 notebook 中的所有代码，以及这个目录中的其余文件，都将被打包并发送到谷歌云执行。 run 方法上的参数指定要使用的执行环境和分发策略（如果有）的详细信息。

提交作业后，您可以转到下一步，通过 Tensorboard 监控作业进度。

In [ ]:
# If you are using a custom image you can install modules via requirements
# txt file.
with open("requirements.txt", "w") as f:
    f.write("tensorflow-cloud\n")

# Optional: Some recommended base images. If you provide none the system
# will choose one for you.
TF_GPU_IMAGE = "gcr.io/deeplearning-platform-release/tf2-cpu.2-5"
TF_CPU_IMAGE = "gcr.io/deeplearning-platform-release/tf2-gpu.2-5"

# Submit a single node training job using GPU.
tfc.run(
    distribution_strategy="auto",
    requirements_txt="requirements.txt",
    docker_config=tfc.DockerConfig(
        parent_image=TF_GPU_IMAGE, image_build_bucket=GCS_BUCKET
    ),
    chief_config=tfc.COMMON_MACHINE_CONFIGS["K80_1X"],
    job_labels={"job": JOB_NAME},
)

# 训练结果
## 重新连接您的 Colab 实例
大多数远程培训工作都是长期运行的。 如果您使用的是 Colab，则可能会在训练结果可用之前超时。

在这种情况下，请重新运行以下部分以重新连接和配置您的 Colab 实例以访问训练结果。
* 导入所需模块
* 项目配置
* 对笔记本进行身份验证以使用您的 Google Cloud 项目

不要重新运行其余的代码。

## 加载Tensorboard
在训练过程中，您可以使用 Tensorboard 查看结果。 请注意，只有在您的训练开始后才会显示结果。 这可能需要几分钟的时间。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $TENSORBOARD_LOGS_DIR

# 加载训练好的模型
训练完成后，您可以从上面指定的 GCS Bucket 中检索模型。

In [ ]:
trained_model = tf.keras.models.load_model(SAVED_MODEL_DIR)
trained_model.summary()